In [1]:
import os
os.chdir('../')
%pwd

'h:\\Personal Projects\\Carbon-Footprint-estimation'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [3]:
from CarbonFootprint.constants import *
from CarbonFootprint.utils.common import read_yaml, create_directories, save_json

In [5]:
from pathlib import Path


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XgBoost 
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=params,
            metric_file_name=Path(config.metric_file_name),
            target_column=list(schema.keys())[0]  # e.g., "CarbonEmission"
        )

        return model_evaluation_config


In [6]:
import os
import pandas as pd
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error,
    r2_score, mean_absolute_percentage_error,
    median_absolute_error
)
from urllib.parse import urlparse
import numpy as np
import joblib

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mse = mean_squared_error(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        ape = np.mean(np.abs((actual - pred) / actual)) * 100  # Average Percentage Error
        medae = median_absolute_error(actual, pred)
        return rmse, mse, mae, r2, ape, medae
    


    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        predicted_qualities = model.predict(test_x)

        (rmse, mse, mae, r2, ape, medae) = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"rmse": rmse, "mse": mse, "mae": mae, "r2": r2, "ape": ape, "medae": medae}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2025-05-13 21:32:13,508: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-13 21:32:13,510: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-13 21:32:13,512: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-13 21:32:13,513: INFO: common: created directory at: artifacts]
[2025-05-13 21:32:13,514: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-13 21:32:13,696: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
